In [6]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display
from bs4 import BeautifulSoup
from openai import OpenAI

In [82]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print('couldn\'t catch API key')
model = 'gpt-4o-mini'
openai = OpenAI()

In [55]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self,url):
        self.url =  url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else ''
        if soup.body:
            for irrelevent in soup.body(["script", "style", "img", "input"]):
                irrelevent.decompose()
            self.text = soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = ''

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def website_content(self):
        return (f'The page title is:{self.title}\nThe content is:\n{self.text}')

In [65]:
#a = Website('https://www.nytimes.com/international/')
a = Website('https://edwarddonner.com')
print(a.title)

Home - Edward Donner


In [58]:
#print(a.website_content())

In [120]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

In [124]:
def get_links_user_prompt(website):
    user_prompt = "Here are the links:\n"
    user_prompt += "\n".join(website.links)
    user_prompt+="\nReturn only the most relevant ones in this JSON format:\n"
    user_prompt+="""{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
    }"""
    return user_prompt

In [125]:
def get_relevant_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=model, 
        messages=[
        {'role': 'system', 'content': link_system_prompt},
        {'role': 'user', 'content': get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
get_relevant_links('https://huggingface.co')